## DOCKER IMAGE
- local-ai: localai/localai:v2.8.2-cublas-cuda11 localai/localai:master-cublas-cuda11 , https://localai.io/basics/getting_started/
- node: node:hydrogen-bullseye
- anythingllm: mintplexlabs/anythingllm
- ollama: ollama/ollama:latest

In [108]:
%%bash
ps -ef |grep 'ollama' | awk '{print $2}' | xargs kill -9
export SINGULARITYENV_OLLAMA_HOST="$(hostname -s):11434"
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
mkdir -p ${DIR}/ollama_models
ml libs/singularity/3.10.2
singularity instance stop ollama
singularity instance start --no-home --nv -B /work -B ${DIR}/ollama_models:$HOME/.ollama /work/u00cjz00/nvidia/ollama_latest.sif ollama
sleep 5
nohup singularity exec instance://ollama ollama serve  > ./ollama.log 2>&1 &

kill: sending signal to 112968 failed: No such process
Error for command "stop": no instance found

Usage:
  singularity [global options...] instance stop [stop options...] [instance]

Run 'singularity --help' for more detailed usage information.
INFO:    instance started successfully


In [ ]:
%%bash 
# download model
export SINGULARITYENV_OLLAMA_HOST="$(hostname -s):11434"
/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity exec instance://ollama ollama pull phi

In [ ]:
%%bash 
# copy model
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
echo "FROM /work/u00cjz00/slurm_jobs/github/models/Taiwan-LLM-7B-v2.1-chat-Q8_0.gguf" > ${DIR}/ollama_models/Modelfile_demo
export SINGULARITYENV_OLLAMA_HOST="$(hostname -s):11434"
/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity exec instance://ollama ollama create Taiwan-LLM-7B-v2.1-chat-Q8_0 -f $HOME/.ollama/Modelfile_demo

In [111]:
# stop ollama
!ps -ef |grep 'ollama' | awk '{print $2}' | xargs kill -9

kill: sending signal to 120514 failed: No such process


## 2. 下載 local-ai 進行模型佈屬
https://localai.io/basics/getting_started/

https://dev.to/worldlinetech/introducing-localai-4gg1

In [106]:
%%bash
rm -rf LocalAI
git clone https://github.com/mudler/LocalAI.git
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
mkdir -p ${DIR}/myLocalAI/models ${DIR}/myLocalAI/images
cp ${DIR}/LocalAI/.env ${DIR}/myLocalAI/.env

Cloning into 'LocalAI'...


In [79]:
%%bash
port=12346
model="phi-2"
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
export SINGULARITYENV_env_file=".env"
ml libs/singularity/3.10.2
ps -ef |grep 'local-ai' | awk '{print $2}' | xargs kill -9
nohup singularity exec --nv -B /work -B /work/u00cjz00/slurm_jobs/github/local-ai_model:/models -B ${DIR}/myLocalAI/images:/tmp/generated/images /work/u00cjz00/nvidia/localai_v2.8.2-cublas-cuda11.sif \
/build/local-ai --address $(hostname -s):${port} --models-path /models --models ${model} > ./local-ai.log 2>&1 &

kill: sending signal to 208976 failed: No such process


In [6]:
# check job
!ps -ef |grep 'local-ai' 

g00cjz00 117552  48250  0 21:00 pts/2    00:00:00 /bin/bash -c ps -ef |grep 'local-ai'
g00cjz00 117554 117552  0 21:00 pts/2    00:00:00 grep local-ai


In [78]:
# kill job
!ps -ef |grep 'local-ai' | awk '{print $2}' | xargs kill -9

kill: sending signal to 208915 failed: No such process


## 2. 下載 anything-llm 進行編譯

In [100]:
%%bash
rm -rf anything-llm
#git clone https://github.com/Mintplex-Labs/anything-llm.git
git clone https://github.com/c00cjz00/anything-llm

Cloning into 'anything-llm'...


In [101]:
%%bash
cat << \EOF >  demo.script

DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
echo "fix"
sed -i "s/5.3.0/5.3.1/g" ${DIR}/anything-llm//server/package.json

echo "Rebuilding Frontend"
cd ${DIR}/anything-llm/frontend && yarn && yarn build

echo "Copying to Sever Public"
rm -rf ${DIR}/anything-llm/server/public
cp -r ${DIR}/anything-llm/frontend/dist ${DIR}/anything-llm/server/public

echo "Installing collector dependencies"
cd ${DIR}/anything-llm/collector && yarn

echo "Installing server dependencies & running migrations"
cd ${DIR}/anything-llm/server && yarn
cd ${DIR}/anything-llm/server && npx prisma migrate deploy --schema=./prisma/schema.prisma
cd ${DIR}/anything-llm/server && npx prisma generate

echo ".env"
touch ${DIR}/anything-llm/server/.env

echo "ssl"
cp -rf ${DIR}/anything-llm-patch/server/ssl ${DIR}/anything-llm/server/ssl
cp ${DIR}/anything-llm-patch/server/env ${DIR}/anything-llm/server/.env
EOF

chmod 755 demo.script

In [ ]:
%%bash
ml libs/singularity/3.10.2
singularity exec -B /work /work/u00cjz00/nvidia/node_hydrogen-bullseye.sif ./demo.script

## 啟動  anything-llm

In [110]:
%%bash
ml libs/singularity/3.10.2
export SINGULARITYENV_STORAGE_DIR="/app/server/storage"
nohup singularity run -B anything-llm:/app /work/u00cjz00/nvidia/anythingllm_latest.sif > ./anything-llm.log 2>&1 &

In [59]:
!/usr/sbin/lsof -i -P -n |grep node

## 連線

In [98]:
%%bash
node_hostname=$(hostname -s)
node_ip=$(cat /etc/hosts |grep "$(hostname -a)" | awk '{print $1}')
# PORT
noed_port_genai=$(python -c "import socket; s = socket.socket(socket.AF_INET, socket.SOCK_STREAM); s.bind(('', 0)); addr = s.getsockname(); s.close(); print(addr[1])")
noed_port_genai=3001

# PROXY
proxy_url=/rstudio/${node_ip}/${noed_port_genai}
# URL
https_url=https://node01.biobank.org.tw${proxy_url}/

# SSH FORWARDING
ssh_cmd="ssh -o StrictHostKeyChecking=no -o TCPKeepAlive=yes -o ServerAliveCountMax=20 -o ServerAliveInterval=15 -NfL ${noed_port_genai}:${node_hostname}:${noed_port_genai} $(whoami)@t3-c4.nchc.org.tw"; 
echo "SSH:"
echo ${ssh_cmd}
echo ""
ssh_cmd="ssh -L ${noed_port_genai}:${node_hostname}:${noed_port_genai} $(whoami)@t3-c4.nchc.org.tw"
echo "SSH:"
echo ${ssh_cmd}
echo ""
echo "URL: http://localhost:${noed_port_genai}"
echo ""

SSH:
ssh -o StrictHostKeyChecking=no -o TCPKeepAlive=yes -o ServerAliveCountMax=20 -o ServerAliveInterval=15 -NfL 3001:gpn3002:3001 g00cjz00@t3-c4.nchc.org.tw

SSH:
ssh -L 3001:gpn3002:3001 g00cjz00@t3-c4.nchc.org.tw

URL: http://localhost:3001



## DELETE JOB

In [8]:
!ps -ef |grep node | awk '{print $2}' | xargs kill -9

kill: sending signal to 7340 failed: Operation not permitted
kill: sending signal to 8598 failed: Operation not permitted
kill: sending signal to 117599 failed: No such process
